In [2]:
from Bio import SeqIO

Le projet suivant est de reproduire un algorithme de mapping de reads sur génome. 
Les caractéristiques de la technnique de séquençage n'est pas prise en compte. L'objectif est avec un read sans erreures de lecture, on essaye de mapper ce read sur un genome. 

Le projet est composé en plusieurs parties : 

Etape_1 : Importation des reads et du genome (afin que les temps de calcul soit acceptables pour que je vous montre les résultats , je ne prendrai que 15 reads et une partie du génome (600 000 nb))

Etape_2 : Algorithme DC3 , l'agorithme DC3 permet de trouver le suffixe array d'un string pour faciliter sa transformation en Burrow wheeler. (je vous ai mit une explication sur le github, si vous êtes intéressés comment ça marche)

Etape_3 : conversion du suffixe array en BWT(burrow wheeler transformation)

Etape_4 : Mapper les reads sur le génome 

Etape_5 : Mapper les reads sur le génome en supposant que le génome a des mutations. 

Etape_1 : Importation des données avec la librairie Bio

In [3]:
Liste_reads = []
i = 0

#on récupère les reads présents dans notre fichier fasta et on stock dans une liste
for seq_record in SeqIO.parse("READS.fq" , "fastq") :
    if i <= 20 : ### A CHANGER SI ON VEUT PLUS DE READS 
        Liste_reads.append(str(seq_record.seq))
        i+= 1
        continue
    break

Liste_genome = []
#on récupère les séquences du génome dans notre fichier fasta et on stock dans une liste
for seq_record in SeqIO.parse("GENOMES.fna" , "fasta") : 
    Liste_genome.append(str(seq_record.seq))
    break

Liste des fonctions pour l'étape 2 (Algorithme DC3)

In [4]:
#Conversion de notre string en une suite de numério ASCII (permet de donner un ordre à notre string)
#exemple : ATCGA == > 12341
def transformation_ASCII(string,correspondance_ASCII) : 
    string_ASCII = []

    for i in range (len(string)) : 
        string_ASCII.append(correspondance_ASCII[string[i]])
    return string_ASCII


# Dans l'agorithme DC3 , R1_R2 correspond aux index dont index % 3 == 1 et index % 3 == 2

# Cette fonction permet de créer une liste qui va retrouver les index de R1_R2 et de R0 
def Trouver_P1_P2(sequence_ASCII) : 
        P0 = []
        P1_P2 = []
        for i in range(1,len(sequence_ASCII)-2,3) : 
             P1_P2.append(i)
        for j in range(2,len(sequence_ASCII)-2,3) : 
             P1_P2.append(j)
        for k in range(0,len(sequence_ASCII)-2,3) : 
             P0.append(k)
        return P1_P2 , P0

# Cette fonction permet de créer une Liste qui comporte les valeurs de R1_R2

def trouver_R1_R2(P1_P2,sequence_ASCII) : 
        R1_R2 = []
        for i in range (len(P1_P2)) : 
            R1_R2.append([])
            R1_R2[i].append(sequence_ASCII[P1_P2[i]])
            R1_R2[i].append(sequence_ASCII[P1_P2[i]+1])
            R1_R2[i].append(sequence_ASCII[P1_P2[i]+2])
        return R1_R2

# L'algorithme radix sort est un algorithme de triage de nombres. C'est un moyen efficace et rapide de trier. 
# radix sort trie digit par digit. 
#radix sort pour trier digit par digit va utilise l'algorithme counting sort

def Radix_sort(R1_R2_sort,R1_R2,Index,Index_R1_R2) : 
        counter = 2
        Index_sort = Index.copy()
        Index_sort_R1_R2 = Index_R1_R2.copy()
        digit_max = max(R1_R2[0])
        for i in range (1,len(R1_R2)) : 
            if digit_max < max(R1_R2[i]) : 
                digit_max = max(R1_R2[i])
        R1_R2_sort,Index_sort,Index_sort_R1_R2 = countingsort(counter,R1_R2_sort,R1_R2,Index,Index_sort,digit_max,Index_R1_R2,Index_sort_R1_R2)
        return R1_R2_sort , Index_sort , Index_sort_R1_R2

#counting sort permet de trier avec une table de comptage puis une somme cumulé de ces comptages

def countingsort(counter,R1_R2_sort,R1_R2,Index,Index_sort,digit_max,Index_R1_R2,Index_sort_R1_R2) :    
        comptage = []
        for i in range(int(digit_max)+1) : 
            comptage.append(0)
        for i in range(len(R1_R2)) : 
            comptage[int(R1_R2[i][counter])] += 1
        for i in range (1,len(comptage)) : 
            comptage[i] = comptage[i]+comptage[i-1]
        for i in range(len(R1_R2)-1,-1,-1) :
            R1_R2_sort[comptage[int(R1_R2[i][counter])] - 1] = R1_R2[i]
            Index_sort[comptage[int(R1_R2[i][counter])] - 1] = Index[i]
            Index_sort_R1_R2[comptage[int(R1_R2[i][counter])] - 1] = Index_R1_R2[i]
            comptage[int(R1_R2[i][counter])] -= 1
        counter -= 1

        R1_R2 = R1_R2_sort.copy()
        Index = Index_sort.copy()
        Index_R1_R2 = Index_sort_R1_R2.copy()
       
        if counter > -1 : 
            R1_R2_sort, Index_sort , Index_sort_R1_R2 = countingsort(counter,R1_R2_sort,R1_R2,Index,Index_sort,digit_max,Index_R1_R2,Index_sort_R1_R2)
            return R1_R2_sort , Index_sort , Index_sort_R1_R2
        return R1_R2_sort , Index_sort , Index_sort_R1_R2

# Cette fonction permet de faire une liste d'ordre/similitude. C'est à dire que chaque élement de R1_R2
#va être attribuer un nombre selon son ordre de gauche à droite dans la liste 
#Les élements identiques ont le même nombre

def order_R1_R2(R1_R2_sort) :
        check = True
        order = []
        order.append(1)
        for i in range(1,len(R1_R2_sort)) : 
            if R1_R2_sort[i] == R1_R2_sort[i-1] :  
                order.append(order[i-1])
                check = False
            else : 
                order.append(order[i-1]+1)
        return order , check

#Cette fonction permet de trouver le "T" selon l'algorithme DC3
#A partir de la liste index de R1_R2, dans le même ordre , on fait une liste T qui pour l'index correspond R1_R2
#remplace par l'ordre qui a été attribué après radix sort.

def T(orderR , Index_sort_R1_R2) :
            T_final = orderR.copy()
            for i in range (len(Index_sort_R1_R2)) : 
                T_final[Index_sort_R1_R2[i]] = orderR[i]
            return T_final


# A partir du "T" , il est possible de retrouver R0. Comme nous avons les index %3 == 1 et 2 
#les seules index restant ne peuvent que être index % 3 == 0 , l'information est stocké indirectement

def creation_R0(Index_sort,P0,T) : 
    R0 = []
    for i in range(len(P0)) : 
        R0.append([])
        R0[i].append(int(T[P0[i]]))
        R0[i].append(P0[i])
    return R0

#la structure de données est différentes entre le trie de R1_R2 et celui de R_0.
#Une autre version adapté à Radix sort est donc utilisé 

def Radix_sort_0(R0_sort,R0,P0) : 
        counter = 1
        P0_sort = P0.copy()
        digit_max = max(R0[0])
        for i in range (1,len(R0)) : 
            if digit_max < max(R0[i]) : 
                digit_max = max(R0[i])
        R0_sort,P0_sort = countingsort_0(counter,R0_sort,R0,P0,P0_sort,digit_max)
        return R0_sort , P0_sort 

#Counting sort adapté à la fonction Radix_sort_R0

def countingsort_0(counter,R0_sort,R0,P0,P0_sort,digit_max) :    
        comptage = []
        for i in range(int(digit_max)+1) : 
            comptage.append(0)
        for i in range(len(R0)) : 
            comptage[int(R0[i][counter])] += 1
        for i in range (1,len(comptage)) : 
            comptage[i] = comptage[i]+comptage[i-1]
        for i in range(len(R0)-1,-1,-1) :
            R0_sort[comptage[int(R0[i][counter])] - 1] = R0[i]
            P0_sort[comptage[int(R0[i][counter])] - 1] = P0[i]
            comptage[int(R0[i][counter])] -= 1
        counter -= 1

        R0 = R0_sort.copy()
        P0 = P0_sort.copy()
    
        if counter > -1 : 
            R0_sort, P0_sort = countingsort_0(counter,R0_sort,R0,P0,P0_sort,digit_max)
            return R0_sort , P0_sort
        return R0_sort , P0_sort

#Permet de retrouver les "orders" de R0, comme ce que nous avons fait avec R1_R2 

def function_order_R0(R0_sort) :
        check = True
        order_R0 = []
        order_R0.append(1)
        for i in range(1,len(R0_sort)) : 
            if R0_sort[i][0] == R0_sort[i-1][0] :  
                order_R0.append(order_R0[i-1])
                check = False
            else : 
                order_R0.append(order_R0[i-1]+1)

        return order_R0 , check

# L'algorithme DC3
  
def DC3_suffix_array(sequence_ASCII) : 
   
    sequence_ASCII.append(0)
    sequence_ASCII.append(0)
    sequence_ASCII.append(0) # on ajoute les nombres sentinelles (dominant par rapport à tous)

    P1_P2 , P0 = Trouver_P1_P2(sequence_ASCII) # on trouve Les index avec modulo 3

    R1_R2 = trouver_R1_R2(P1_P2,sequence_ASCII) # dans les index , on met les 3 nombres qui suivent l'index
    #R1_R2 comporte des Listes dans une liste 
    Index_R1_R2 = []
    R1_R2_sort = []
    for i in range(len(R1_R2)) : # permet de former la future liste qui contiendra nos R1_R2 sort 
        R1_R2_sort.append([])
    for i in range(len(R1_R2)) : # permet d'avoir les index de R1_R2 avant sort puis de les répartir selon le trie 
        Index_R1_R2.append(i) 

    Index = P1_P2.copy()
    
    R1_R2_sort,Index_sort,Index_sort_R1_R2 = Radix_sort(R1_R2_sort,R1_R2,Index,Index_R1_R2) #Radix sort de R1_R2

    order_R , check  = order_R1_R2(R1_R2_sort) # on trouve les ordres du trie , le but est de n'avoir aucune répétition d'ordre

    if check == False : # si check == false , il y a des répétions dans nos ordres, on réitère une nouvelle fois pour n'avoir aucune répétition
        T_final = T(order_R , Index_sort_R1_R2) #on trouve le T qui correspond à l'input
        T_prime = DC3_suffix_array(T_final)  # A partir de ce T , on recommence l'algorithme pour trouver son T'
        Index_sort = []
        
        for i in range(len(T_prime)) : 
            Index_sort.append(P1_P2[T_prime[i]]) # on récupère ses index
        
    # le check est True donc on a pas de doublon dans notre T 

    #Toutes les itérations de T sont terminées , il faut maintenant retrouver R0

    R0 = creation_R0(Index_sort,P0,sequence_ASCII) # création de R0

    R0_sort = []
    for i in range(len(R0)) : # préparation de la future liste contenant R0 sort 
            R0_sort.append([])

    R0_sort , P0_sort_index = Radix_sort_0(R0_sort,R0,P0) # on trie R0

    order_R0 , check_R0 = function_order_R0(R0_sort) # on trouve les orders de R0

# création d'un dictionnaire afin de trouver plus rapidement les index, si on ne stocke pas dans la mémoire
#le temps de calcul sera beaucoup trop long.
# Le code suivant permet à partir des index et des ordres de R0 , de retrouver L'ordre exacte des index et les mettre dans une liste
    index_dict = {value: index for index, value in enumerate(Index_sort)} 
    if check_R0 == False : 
        i = 0 
        P0_sort_index = []
        while i < len(order_R0) : 
            j = 1
            if j + i < len(order_R0) : 
                while order_R0[i] == order_R0[i+j] : 
                    j+= 1
                    if j+i >= len(order_R0) : 
                        break
                if j != 1 : 
                    test = []
                    counter = 0
                    for k in range(i,i+j) : 
                        test.append([])
                        h = index_dict[R0_sort[k][1]+1]
                        test[counter].append(h)
                        test[counter].append(R0_sort[k][1])
                        counter+=1
                    test = sorted(test,key=lambda x:x[0])
                    for k in range(len(test)) : 
                        P0_sort_index.append(test[k][1])
                    i += j 
                    if i+j >= len(order_R0) : 
                        break
                   
                else : 
                    
                    P0_sort_index.append(R0_sort[i][1])
                    i+=1
                    if i+j >= len(order_R0) : 
                        break
        if i < len(R0_sort) : 
             P0_sort_index.append(R0_sort[i][1])
             i+= 1
       
    # Dernière étape de l'algorithme DC3 , il faut merge R0 et R1_R2 maintenant que nous les avons. 

    Index_1_2_3 = []
    iterator_a = 0
    iterator_b = 0
    print("T : " , len(sequence_ASCII)) 
    print("combin :  " , len(P0_sort_index) + len(Index_sort)) # longueur des index de tout les modulos , une différence de 2 car dans les 3 zéros ajouter au départ , seulement 1 est utilisé dans l'algo

# code ici permet de merge R0 et R1_R2
# Nos deux listes R1_R2 et R0 sont parcourus de gauche à droite. Il y a une comparaison de leurs ordres et valeurs 
#la valeur la plus forte est ajouté dans la merged_liste puis retirer. L'objectif est d'ajouter tout les élements de R1_R2 et R_0 dans la nouvelle liste 

    while len(Index_1_2_3) < (len(sequence_ASCII)-2) : 
        
        if (iterator_a) < len(P0_sort_index) and (iterator_b) < len(Index_sort) : 
            
            if sequence_ASCII[P0_sort_index[iterator_a]] < sequence_ASCII[Index_sort[iterator_b]] :
                Index_1_2_3.append(P0_sort_index[iterator_a])
                iterator_a+= 1
                
                continue
            if sequence_ASCII[P0_sort_index[iterator_a]]  > sequence_ASCII[Index_sort[iterator_b]] :
                Index_1_2_3.append(Index_sort[iterator_b])
                iterator_b+= 1
                continue
            if sequence_ASCII[P0_sort_index[iterator_a]]  == sequence_ASCII[Index_sort[iterator_b]] :
                if Index_sort[iterator_b]%3 == 1 : 
                    b = 0
                    a = 0
                    b = index_dict[Index_sort[iterator_b]+1]
                    a = index_dict[P0_sort_index[iterator_a]+1]
                    if b < a : 
                        Index_1_2_3.append(Index_sort[iterator_b])
                        iterator_b+= 1
                        continue
                    else : 
                        Index_1_2_3.append(P0_sort_index[iterator_a])
                        iterator_a+= 1
                        continue
                else : 
                   
                    if sequence_ASCII[P0_sort_index[iterator_a]+1]  < sequence_ASCII[Index_sort[iterator_b]+1] : 
                        Index_1_2_3.append(P0_sort_index[iterator_a])
                        iterator_a+= 1
                        continue
                    if sequence_ASCII[P0_sort_index[iterator_a]+1]  > sequence_ASCII[Index_sort[iterator_b]+1] :
                        Index_1_2_3.append(Index_sort[iterator_b])
                        iterator_b+= 1
                        continue
                    else : 
                        b = 0
                        a = 0
                        b = index_dict[Index_sort[iterator_b]+2]
                        a = index_dict[P0_sort_index[iterator_a]+2]
                        if b < a : 
                            Index_1_2_3.append(Index_sort[iterator_b])
                            iterator_b+= 1
                            continue
                        else : 
                            Index_1_2_3.append(P0_sort_index[iterator_a])
                            iterator_a+= 1
                            continue
        
           
        while (iterator_a) != len(P0_sort_index) :
            Index_1_2_3.append(P0_sort_index[iterator_a])
            iterator_a+= 1
        while (iterator_b) != len(Index_sort) : 
            
            Index_1_2_3.append(Index_sort[iterator_b])
            iterator_b+=1
   
    Index_1_2_3.pop(0)
    del index_dict
    
    return Index_1_2_3 # Index _1,2,3 signifie la merged liste de R0_R1 et R2

fonctions pour les étapes suivantes BWT , BWT inverse , mapping sans mutation et avec mutation 

In [5]:
def burrow_whealer(sequence_genome , suffix_array) : # permet d'obtenir la BWT du genome à partir de son suffixe array
    str_burrow_whealer = ""
    for i in range(len(suffix_array)) : 
         str_burrow_whealer += str(sequence_genome[suffix_array[i]-1])
    return str_burrow_whealer

def counter(BWT) : # permet de trouver l'apparition du premier A,C,G,T dans la suite cumulée de chaque lettre 
    #exemple ACCGAGT ===> AATCCGG
    A = BWT.count("A")
    C = BWT.count("C")
    G = BWT.count("G")
    T = BWT.count("T")
    BWT_order = ""
    for i in range(A) : 
        BWT_order = BWT_order + "A"
    for i in range(C) : 
        BWT_order = BWT_order + "C"
    for i in range(G) : 
        BWT_order = BWT_order + "G"
    for i in range(T) : 
        BWT_order = BWT_order + "T"
    
    First_T = (len(BWT_order) - T)+1
    First_G = First_T - G
    First_C = First_G - C
    First_A = First_C - A
    return BWT_order,First_A,First_C,First_G,First_T

def data_counter(BWT) : # permet de compter les lettres dans la BWT afin de les retrouver selon leurs numéros 
    data_A = [0]
    data_C = [0]
    data_G = [0]
    data_T = [0]
    for i in range(len(BWT)) : 
        counterA = 0
        counterC = 0
        counterG = 0 
        counterT = 0 
        if BWT[i] == "A" : 
            counterA = 1
            
        if BWT[i] == "C" : 
            counterC = 1
        if BWT[i] == "G" : 
            counterG = 1  
        if BWT[i] == "T" : 
            counterT = 1    
        data_A.append(data_A[i]+counterA)
        data_C.append(data_C[i]+counterC)
        data_G.append(data_G[i]+counterG)
        data_T.append(data_T[i]+counterT)
    
    return data_A,data_C,data_G,data_T

#La fonction Ici correspond au "pointeur B" , il permet de suivre et reconstruire le string voulu 
def recherche_pointeur_B(pointeur_B,pointeur_max,counter,i,pointeur_start,selected_read) : 
        global BWT
        if counter <= pointeur_max : 
            for k in range(pointeur_start,pointeur_max) : 
                    counter +=1
                    if BWT[k] == selected_read[-i-1] : 
                        pointeur_B = k 
            pointeur_B = recherche_pointeur_B(pointeur_B,pointeur_max,counter,i,pointeur_start,selected_read)
        return pointeur_B

# Cette fonction permet de retrouver le string dans la BWT , le pointeur A est inclue dans cette fonction 
def recherche_string_BWT(selected_read) : 
        global dictionnaire_FIRST
        global dictionnaire_DATA
        global BWT
        i = 1
        pointeur_start = dictionnaire_FIRST[selected_read[-i]] 
        pointeur_max = pointeur_start - 1  + dictionnaire_DATA[selected_read[-i]][-1] +1
        while i < len(selected_read) :  
            pointeur_A = -1
            pointeur_B = -1
            counter = pointeur_start-1
            
            for j in range(pointeur_start,pointeur_max) : 
                counter+= 1
                if BWT[j] == selected_read[-i-1] : 
                    pointeur_A = j
                    break
            pointeur_B = recherche_pointeur_B(pointeur_B,pointeur_max,counter,i,pointeur_start,selected_read)
        
            if pointeur_A != -1 and pointeur_B == -1 : 
                pointeur_B = pointeur_A 
        
            if pointeur_A != -1 : 
                pointeur_start = dictionnaire_DATA[selected_read[-i-1]][pointeur_A+1]+ dictionnaire_FIRST[selected_read[-i-1]] - 1
                pointeur_max = dictionnaire_DATA[selected_read[-i-1]][pointeur_B+1] + dictionnaire_FIRST[selected_read[-i-1]]
                i+=1
                continue
            else : 
                return
        search_prefix = []

        for i in range(pointeur_start,pointeur_max) : 
            search_prefix.append(i)
        return search_prefix

# Fonction à partir d'ici vont correspondre à la recherche avec mutation 


# Permet de transformer nos reads en Kmers
def transformation_kmers(selected_read,Kmers) : 
    Liste_Kmers = []
    for i in range(len(selected_read)-Kmers+1) : 
        Liste_Kmers.append(selected_read[i:i+Kmers])
    return Liste_Kmers,Kmers
# même chose que le pointeur B 
def recherche_pointeur_B(pointeur_B,pointeur_max,counter,i,pointeur_start,selected_word) : 
            if counter <= pointeur_max : 
                for k in range(pointeur_start,pointeur_max) : 
                        counter +=1
                        if BWT[k] == selected_word[-i-1] : 
                            pointeur_B = k 
                pointeur_B = recherche_pointeur_B(pointeur_B,pointeur_max,counter,i,pointeur_start,selected_word)
            return pointeur_B

# même chose également

def recherche_string_BWT(selected_kmers) : 
                i = 1
                pointeur_start = dictionnaire_FIRST[selected_kmers[-i]] 
                pointeur_max = pointeur_start - 1  + dictionnaire_DATA[selected_kmers[-i]][-1] +1
                while i < len(selected_kmers) :  
                    pointeur_A = -1
                    pointeur_B = -1
                    counter = pointeur_start-1
                    
                    for j in range(pointeur_start,pointeur_max) : 
                        counter+= 1
                        if BWT[j] == selected_kmers[-i-1] : 
                            pointeur_A = j
                            break
                    pointeur_B = recherche_pointeur_B(pointeur_B,pointeur_max,counter,i,pointeur_start,selected_kmers)
                
                    if pointeur_A != -1 and pointeur_B == -1 : 
                        pointeur_B = pointeur_A 
                
                    if pointeur_A != -1 : 
                        pointeur_start = dictionnaire_DATA[selected_kmers[-i-1]][pointeur_A+1]+ dictionnaire_FIRST[selected_kmers[-i-1]] - 1
                        pointeur_max = dictionnaire_DATA[selected_kmers[-i-1]][pointeur_B+1] + dictionnaire_FIRST[selected_kmers[-i-1]]
                        i+=1
                        continue
                    else : 
                        
                        return
                
                search_prefix = []
                for i in range(pointeur_start,pointeur_max) : 
                    search_prefix.append(i)
                return search_prefix

#permet de chercher notre Kmers dans la bwt

def Recherche_Position_Kmers(Liste_Kmers,Liste_positions_Kmers) : 
        global Suffixe_array

        for i in range(len(Liste_Kmers)) : 
            selected_kmers = Liste_Kmers[i]                        
            search_prefix = recherche_string_BWT(selected_kmers)

            if search_prefix : 

                position_pattern = []
                for i in range(len(search_prefix)) : 
                    position_pattern.append(Suffixe_array[search_prefix[i]])
                Liste_positions_Kmers.append(position_pattern)
            else : 
                Liste_positions_Kmers.append([])

def recherche_mutation_suite(target,Liste_positions_Kmers,j) : 
        for k in range(len(Liste_positions_Kmers[j])) : 
                if target+1 == Liste_positions_Kmers[j][k] : 
                    target = Liste_positions_Kmers[j][k]
                    return True,target
        return False,target

#fonction permettant d'inclure la possibilité d'avoir des mutations en incluant nos kmers

def recherche_mutation(Liste_positions_Kmers,Kmers) : 
        global position_start

        for i in range(len(Liste_positions_Kmers[0])) : 
            counter = 0
            target = Liste_positions_Kmers[0][i]
            for j in range(1,len(Liste_positions_Kmers)) :       
                check , target = recherche_mutation_suite(target,Liste_positions_Kmers,j)
                if check == True : 
                    continue
                else : 
                    if j < Kmers or j > len(Liste_positions_Kmers) - Kmers : 
                        break 
                    else : 
                        if counter <= Kmers  : 
                            counter+=1
                            target+= 1
                            continue 
                        else : 
                            break
                    
            if check == True : 
                position_start.append(Liste_positions_Kmers[0][i])
        return position_start


Fonction Main - Pour l'algorithme DC3

In [6]:
sequence_genome = Liste_genome[0] # beaucoup de genomes donc on sélectionne juste le premier 

sequence_genome_string = "" # notre genome est composé de a , A , c , T etc. on veut que tout soit en majuscule. 

dictionnaire_conversion = {"A" : "A" , "a" : "A" , "C" : "C" , "c" : "C" , "G" : "G" , "g" : "G" , "T" : "T" , "t" : "T" , "N" : "N" , "n" : "N"}
for i in range(len(sequence_genome)) : 
    sequence_genome_string += dictionnaire_conversion[sequence_genome[i]]

sequence_geneome_complementaire = "" # on veut obtenir la même séquence du génome en complémentaire. 

dictionnaire_conversion_complementaire = {"A" : "T" , "a" : "T" , "C" : "G" , "c" : "G" , "G" : "C" , "g" : "C" , "T" : "A" , "t" : "A", "N" : "N" , "n" : "N"}
for i in range(1,len(sequence_genome)+1) : 
    sequence_geneome_complementaire += dictionnaire_conversion_complementaire[sequence_genome[-i]]

correspondance_ASCII = {"A" : 1 , "C" : 2 , "G" : 3 , "T" : 4 , "N" : 5 , "a"  : 1 , "c" : 2 , "g" : 3 , "t" : 4}

selected_genome_version = sequence_geneome_complementaire # Choisir ici si on veut chercher dans le sens direct ou complémentaire

sequence_genome_ASCII = transformation_ASCII(selected_genome_version,correspondance_ASCII) # on convertit en version ASCII

Suffixe_array = DC3_suffix_array(sequence_genome_ASCII) # on trouve le suffixe array pour reconstruire la BWT

BWT = burrow_whealer(selected_genome_version,Suffixe_array) # on reconstruit la BWT

#préparation de notre BWT afin de pouvoir chercher nos strings dans le génome en version BWT
Index_recherche_0 = Suffixe_array.index(0)
BWT = BWT[:Index_recherche_0] + "$" + BWT[Index_recherche_0+1:]
BWT = sequence_geneome_complementaire[-1] + BWT
Suffixe_array.insert(0,len(Suffixe_array))


T :  56265
combin :   56263
T :  84395
combin :   84393
T :  126591
combin :   126589
T :  189885
combin :   189883
T :  284826
combin :   284824
T :  427237
combin :   427235
T :  640854
combin :   640852


Fonction Main pour le mapping de reads sans mutation

In [7]:

BWT_order,First_A,First_C,First_G,First_T = counter(BWT) # on récupère les index de la suite cumulé des lettres

data_A,data_C,data_G,data_T = data_counter(BWT) # indexation des lettres de notre BWT

dictionnaire_FIRST = { "A" : First_A , "C" : First_C , "G" : First_G , "T" : First_T}
dictionnaire_DATA = {"A" : data_A , "C" : data_C , "G" : data_G , "T" : data_T}
resultat_final = []

# RECHERCHE UNIQUEMENT DES READS SANS MUTATION / IL FAUT QUE CA SOIT SANS ERREUR 

for p in range(len(Liste_reads)) :  # sélectionne un read de la liste 
    selected_read = Liste_reads[p]         
    search_prefix = recherche_string_BWT(selected_read) # rechercher un string dans la bwt 
    if search_prefix : # si il y en a de trouvé , on les ajoute dans une nouvelle liste , avec le même index que sa position dans Liste_reads

        position_pattern = []

        for i in range(len(search_prefix)) : 
            position_pattern.append(Suffixe_array[search_prefix[i]])
        resultat_final.append(position_pattern)
    else : 
        resultat_final.append([])

print("resultat sans mutation")
print(resultat_final)  

print(resultat_final)
# A retirer si on ne veut pas spam 
i = 0
for position_Liste in resultat_final : 
    if position_Liste : 
        for element in position_Liste : 
            print("A LA POSITION" , element , "LE READ NUMERO" , i)
            print(Liste_reads[i])
            print(selected_genome_version[element:element+100])
    i+=1

resultat sans mutation
[[], [], [], [], [], [], [557971], [], [], [], [], [619112], [], [], [], [33089], [], [], [], [507090], []]
[[], [], [], [], [], [], [557971], [], [], [], [], [619112], [], [], [], [33089], [], [], [], [507090], []]
A LA POSITION 557971 LE READ NUMERO 6
ATAAAATAATAAACTACAACTAAACATTACATATTTATATATGTCACCATTTCTTTATAGGTATATAATCAAAGGAACTATTACATTACACCACGTCATA
ATAAAATAATAAACTACAACTAAACATTACATATTTATATATGTCACCATTTCTTTATAGGTATATAATCAAAGGAACTATTACATTACACCACGTCATA
A LA POSITION 619112 LE READ NUMERO 11
CTTAAGACCTATGTTAGTTACCTTAAGACCTAAGTTACTGTAGCTAGGACCTATATTTGCTAAGTTAAGACCAAACATAGTACCCTAATGACTTATGTTA
CTTAAGACCTATGTTAGTTACCTTAAGACCTAAGTTACTGTAGCTAGGACCTATATTTGCTAAGTTAAGACCAAACATAGTACCCTAATGACTTATGTTA
A LA POSITION 33089 LE READ NUMERO 15
GAAGTGGAATAATAAAGAGGAACTATTAGATAAATTAAAAGAAGAGTGGGAAAATGAGACACATAGTGGTAACATTCACACTAGTGATAGTAACAAAACG
GAAGTGGAATAATAAAGAGGAACTATTAGATAAATTAAAAGAAGAGTGGGAAAATGAGACACATAGTGGTAACATTCACACTAGTGATAGTAACAAAACG
A LA POSITION 507090 LE READ NUMERO 19
A

Fonction Main pour l'algorithme avec mutation

In [8]:

###############################################################################
#RECHERCHE DES READS EN PRENANT EN COMPTE LA POSSIBILITE DE MUTATION 

resultat_final_mutation = []

# même chose pour le mapping mais en prenant en compte les mutations, le nombre de mutation accepté dépend du nombre de kmers
for p in range(len(Liste_reads)) : 
    print("p" , p) # indique à quel reads on est entrain de compute
    selected_read = Liste_reads[p]
    Kmers = 15
    Liste_Kmers,Kmers = transformation_kmers(selected_read,Kmers)

    Liste_positions_Kmers = []
    Recherche_Position_Kmers(Liste_Kmers,Liste_positions_Kmers)
    position_start = []

    if Liste_positions_Kmers : 
        position_start = recherche_mutation(Liste_positions_Kmers,Kmers)
        resultat_final_mutation.append(position_start)
        continue  
    resultat_final_mutation.append([])
    
# A retirer si on ne veut pas spam 
print(resultat_final_mutation)

i = 0
for position_Liste in resultat_final_mutation : 
    if position_Liste : 
        for element in position_Liste : 
            print("A LA POSITION" , element , "LE READ NUMERO" , i)
            print(Liste_reads[i])
            print(selected_genome_version[element:element+100])
    i+=1

p 0
p 1
p 2
p 3
p 4
p 5
p 6
p 7
p 8
p 9
p 10
p 11
p 12
p 13
p 14
p 15
p 16
p 17
p 18
p 19
p 20
[[], [], [], [], [488074], [], [557971], [], [], [177181], [], [619112], [], [], [], [33089], [], [], [], [507090], [319807]]
A LA POSITION 488074 LE READ NUMERO 4
ATGCTTTTTAAAAATGAAGACAGTGGCAATGGTGTTTACGGTTTTACTTATAATAATTTTAGTGATGTTACTATATCTGATATTTTAAATATAAGGAATG
ATGCTTTTTAAAAATGAAGACAGTGGCAATGGTGTTTACTGTTTTACTTATAATAATTTTAGTGATGTTACTATATCTGATATTTTAAATATAAGGAATG
A LA POSITION 557971 LE READ NUMERO 6
ATAAAATAATAAACTACAACTAAACATTACATATTTATATATGTCACCATTTCTTTATAGGTATATAATCAAAGGAACTATTACATTACACCACGTCATA
ATAAAATAATAAACTACAACTAAACATTACATATTTATATATGTCACCATTTCTTTATAGGTATATAATCAAAGGAACTATTACATTACACCACGTCATA
A LA POSITION 177181 LE READ NUMERO 9
TATAATAATAATACTAAATTGTAATAATTATAATATAACAAAAATAAAATAAAAACACTATTATCCATATAAATATATAAATGAAGAACCTCCTTTTGAT
TATAATAATAATACTAAATTGTAATAATTATAATATAACAAAAATAAAATAAAAACACTATAATCCATATAAATATATAAATGAAGAACCTCCTTTTGAT
A LA POSITION 619112 LE READ NUMERO 11
CTTAAGACCTATGTTAGTTA

Référénce pour le fonctionnement de DC3 : Tutoriel de Juha Kärkkäinen et Peter Sanders
DC3 Algorithm Invented/Developed By,
Juha Kärkkäinen
&
Peter Sanders
Tutorial By
Vikas Awadhiya
LinkedIn Profile: https://in.linkedin.com/in/awadhiya-vikas